# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
from collections import OrderedDict
from frozendict import frozendict
import functools

In [2]:
class LruCache(object):
    def __init__(self, max_count):
        self.max_count = max_count
        self.stack = OrderedDict()

    def __getitem__(self, key):
        if key not in self.stack:
            return None
        return self.stack[key]

    def __setitem__(self, key, value):
        if len(self.stack) >= self.max_count:
            self.stack.popitem(last=False)
        self.stack[key] = value

#### Реализуйте декоратор

In [3]:
def cached(max_count):
    cache = LruCache(max_count)
    def getter_func(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = (args, frozendict(kwargs))
            if cache[key] is None:
                cache[key] = func(*args, **kwargs)
            else:
                print("used: ", key)
            return cache[key]
        return wrapper
    return getter_func

#### Проверьте использование декоратора

In [4]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

In [5]:
%time fact(100)

CPU times: user 4.32 ms, sys: 37 µs, total: 4.35 ms
Wall time: 5.18 ms


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [6]:
%time fact(100)

used:  ((100,), <frozendict {}>)
CPU times: user 477 µs, sys: 42 µs, total: 519 µs
Wall time: 417 µs


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

#### Сравните свою реализацию с lru_cache из functools

In [7]:
from functools import lru_cache
@lru_cache(maxsize=20, typed=True)
def fact1(n):
    if n < 2:
        return 1
    return fact(n-1) * n

In [8]:
%time fact1(100)

used:  ((99,), <frozendict {}>)
CPU times: user 0 ns, sys: 420 µs, total: 420 µs
Wall time: 323 µs


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [9]:
%time fact1(100)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10 µs


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

Как можно видеть, значения сильно меняются при перезапуске, но кэширование ускоряет.

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [10]:
def checked(*types):
    def getter_func(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if any([not isinstance(arg, type_arg) for arg, type_arg in zip(args, types)]):
                raise TypeError
            return func(*args, **kwargs)
        return wrapper
    return getter_func
    return decorator

#### Проверьте использование декоратора

In [11]:
from typing import List

@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [12]:
strange_func("a", 5, [1, 2])

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [13]:
def singleton(class_):
    instances = {}
    def getinstance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]
    return getinstance

In [14]:
import sys
from datetime import datetime

@singleton
class Logger:
    def __init__(self, file, format_write="index date time functio_name *args **kwargs result"):
        self.format_write=format_write.split()
        self.file=file
        self.printed_index = 0
        
    def _create_log_str_(self, func_name, args, kwargs, result):
        line = []
        
        for category in self.format_write:
            if category == 'index':
                line.append(self.printed_index)
                self.printed_index += 1
            elif category == 'date':
                line.append(datetime.now().date().strftime("%B %d, %Y"))
            elif category == "time":
                line.append(datetime.now().time().strftime('%H:%M:%S'))
            elif category == "functio_name":
                line.append(func_name)
            elif category == "*args":
                line.append(args)
            elif category == "**kwargs":
                line.append(kwargs)
            elif category == "result":
                line.append(result)
            else:
                raise TypeError
        
        return ' '.join(str(line)) + '\n'
        
    def write(self, func_name, args, kwargs, result):
        line = self._create_log_str_(func_name, args, kwargs, result)
        if line is not None:        
            if isinstance(self.file, str):
                with open(self.file, "a") as file:
                    file.write(line)
            else:
                self.file.write(line)

    def __call__(self, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            self.write(func.__name__, args, kwargs, result)
            return result
        return wrapper

In [15]:
logger = Logger(file=sys.stdout)

In [16]:
@logger
def func_test_int(num):
    return 2*num

@logger
def func_test_str(line):
    return line

In [17]:
func_test_int(3)
func_test_int(4)
func_test_int(5)
func_test_int(6)
func_test_str('abc')
func_test_str('def')
func_test_str('ghi')

[ 0 ,   ' M a r c h   2 9 ,   2 0 1 8 ' ,   ' 1 5 : 5 1 : 3 9 ' ,   ' f u n c _ t e s t _ i n t ' ,   ( 3 , ) ,   { } ,   6 ]
[ 1 ,   ' M a r c h   2 9 ,   2 0 1 8 ' ,   ' 1 5 : 5 1 : 3 9 ' ,   ' f u n c _ t e s t _ i n t ' ,   ( 4 , ) ,   { } ,   8 ]
[ 2 ,   ' M a r c h   2 9 ,   2 0 1 8 ' ,   ' 1 5 : 5 1 : 3 9 ' ,   ' f u n c _ t e s t _ i n t ' ,   ( 5 , ) ,   { } ,   1 0 ]
[ 3 ,   ' M a r c h   2 9 ,   2 0 1 8 ' ,   ' 1 5 : 5 1 : 3 9 ' ,   ' f u n c _ t e s t _ i n t ' ,   ( 6 , ) ,   { } ,   1 2 ]
[ 4 ,   ' M a r c h   2 9 ,   2 0 1 8 ' ,   ' 1 5 : 5 1 : 3 9 ' ,   ' f u n c _ t e s t _ s t r ' ,   ( ' a b c ' , ) ,   { } ,   ' a b c ' ]
[ 5 ,   ' M a r c h   2 9 ,   2 0 1 8 ' ,   ' 1 5 : 5 1 : 3 9 ' ,   ' f u n c _ t e s t _ s t r ' ,   ( ' d e f ' , ) ,   { } ,   ' d e f ' ]
[ 6 ,   ' M a r c h   2 9 ,   2 0 1 8 ' ,   ' 1 5 : 5 1 : 3 9 ' ,   ' f u n c _ t e s t _ s t r ' ,   ( ' g h i ' , ) ,   { } ,   ' g h i ' ]


'ghi'